In [6]:
import os
import zipfile
import random
import shutil
from shutil import copyfile
from os import getcwd
import pathlib
import datetime
from distutils.dir_util import copy_tree

import cv2
from PIL import Image


import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split



In [ ]:
# Most of below is splitting images to use as keras.flow_from_directory
fromDir = "../input/microsoft-catsvsdogs-dataset/PetImages"
toDir = "/kaggle/working/CatsAndDogs/Pet_images_new"

copy_tree(fromDir, toDir)

In [9]:
current_working_dir = os.getcwd()

path_to_images = pathlib.Path(os.path.join(current_working_dir, 'CatsAndDogs','Pet_images_new'))

#current_working_dir = pathlib.Path('..','input','microsoft-catsvsdogs-dataset') #os.getcwd()
#path_to_images = pathlib.Path(os.path.join(current_working_dir, 'PetImages'))
Dogs_dir = os.path.join(path_to_images, 'Dog')
Cats_dir = os.path.join(path_to_images, 'Cat')

print(len(os.listdir(Dogs_dir)))
print(len(os.listdir(Cats_dir)))


dog_image_fnames = os.listdir(Dogs_dir)
cat_image_fnames = os.listdir(Cats_dir)

12501
12501


In [10]:
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join(path_to_images, folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 1590 images


In [11]:
import os

j=0
for catagory in os.listdir(path_to_images):
    sub_dir = os.path.join(path_to_images, catagory)
    for name in os.listdir(sub_dir):
        fpath = os.path.join(sub_dir,name)
        if name.split('.')[1] == 'jpg':
            if os.path.getsize(fpath) <= 0:
                if os.path.isfile(fpath) == False:
                    print(fpath)
                os.remove(fpath)
            elif os.path.getsize(fpath) > 0:
                j +=1
        if name.split('.')[1] != 'jpg':
            print(fpath)
            os.remove(fpath)
print(j)

/kaggle/working/CatsAndDogs/Pet_images_new/Dog/Thumbs.db
/kaggle/working/CatsAndDogs/Pet_images_new/Cat/Thumbs.db
23410


In [ ]:
file_names_of_bad_images = ['5673.jpg','4688.jpg','12376.jpg','1043.jpg','8456.jpg',
                      '9517.jpg','7377.jpg','1773.jpg','8736.jpg','10712.jpg','7564.jpg']

In [12]:
image_count = len(list(path_to_images.glob('*/*.jpg')))
print(image_count)

23410


In [13]:
batch_size = 64  # max 1000 of these images can fit in gpu memory
new_img_size = (200,200)
data_split = 0.2 #0.0005
seed_value = 42

train_it = tf.keras.preprocessing.image_dataset_from_directory(
    path_to_images,
    validation_split=data_split,
    subset="training",
    seed=seed_value,
    image_size=new_img_size,
    batch_size=batch_size)

test_it = tf.keras.preprocessing.image_dataset_from_directory(
    path_to_images,
    validation_split=data_split,
    subset="validation",
    seed=seed_value,
    image_size=new_img_size,
    batch_size=batch_size)

Found 23410 files belonging to 2 classes.
Using 18728 files for training.
Found 23410 files belonging to 2 classes.
Using 4682 files for validation.


In [46]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import sys

In [57]:
from tensorflow.keras.layers import Reshape
def create_model():
    #VGG-4 model with dropout
    model = Sequential()
    model.add(layers.experimental.preprocessing.Rescaling(1./255,input_shape=(new_img_size[0],new_img_size[1],3)))
    model.add(tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'))
    model.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.3))
    # block 1
    model.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))
    # block 2
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))
    # block 3
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))
    # block 4
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))
    
    # block 5
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(MaxPool2D((2, 2)))
    model.add(Dropout(0.2))
    
    # last layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.35))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [59]:
model = create_model()

In [60]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
stop_loss = EarlyStopping(monitor='val_loss', patience=10)
epochs = 50
history = model.fit(
    train_it,
    validation_data=test_it,
    epochs=epochs//2,
    verbose=1,
    callbacks=[stop_loss]
)

Epoch 1/25
293/293 [==============================] - 47s 153ms/step - loss: 1.1230 - accuracy: 0.5371 - val_loss: 0.7427 - val_accuracy: 0.5045
Epoch 2/25
293/293 [==============================] - 45s 151ms/step - loss: 0.6309 - accuracy: 0.6123 - val_loss: 0.5924 - val_accuracy: 0.6653
Epoch 3/25
293/293 [==============================] - 45s 150ms/step - loss: 0.6132 - accuracy: 0.6210 - val_loss: 0.5949 - val_accuracy: 0.6739
Epoch 4/25
293/293 [==============================] - 44s 150ms/step - loss: 0.5847 - accuracy: 0.6464 - val_loss: 0.5820 - val_accuracy: 0.7114
Epoch 5/25
293/293 [==============================] - 45s 151ms/step - loss: 0.5687 - accuracy: 0.6927 - val_loss: 0.6486 - val_accuracy: 0.6820
Epoch 6/25
293/293 [==============================] - 45s 151ms/step - loss: 0.5305 - accuracy: 0.7256 - val_loss: 0.6314 - val_accuracy: 0.6559
Epoch 7/25
293/293 [==============================] - 44s 150ms/step - loss: 0.5004 - accuracy: 0.7444 - val_loss: 0.4925 - val_ac